# Scopo del notebook di Preparation

Lo scopo di questa parte è caricare i dati originali, esplorarli, visualizzarli e prepararli
(es. cleaning e features engineering) per le fasi successive.

# Settings

In [ ]:
# IMPORTAZIONE LIBRERIE

# PACCHETTI DI BASE
import utils
import splitfolders
import pickle
import os
import glob

# PACCHETTI PER L'ANALISI NUMERICA E DI IMMAGINI
import cv2
import numpy as np
import pandas as pd

# VISUALIZZAZIONI
import matplotlib.pyplot as plt
import plotly.express as px

# SKLEARN
from skimage.io import imread
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Costanti

CATEGORIES = ['OK', # non difettose
              'NOK']  # difettose

DIR_ORGINAL_DATA = '../Data/Original/'  # path contenente tutte le categorie di immagini originali
DIR_ORGINAL_N0K_DATA = "../Data/Original/NOK/"  # path contenente le immagine difettose
DIR_ORGINAL_0K_DATA =  "../Data/Original/OK/"  # path contenente le immagine non difettose
DIR_PREPARED_DATA = '../Data/Prepared/'  # path contenente le immagini processate
DIR_TRAIN_TEST_DATA = '../Data/Prepared/TRAIN_TEST' # path contenente i sottoinsiemi di train, test
DIR_TRAIN_DATA = "../Data/Prepared/TRAIN_TEST/train" # path contenente i sottoinsiemi di train
DIR_TEST_DATA = "../Data/Prepared/TRAIN_TEST/val" # path contenente i sottoinsiemi di test

SCALE_PERCENT = 6     # ridimensionamento delle immagini per motivi computazionali di un valore percentuale pari a 6
RANDOM_STATE = 123
scaler = MinMaxScaler()

# Parametri di cropping: creazione di una finestra per il ritagliamento dell'immagine
x=800
w=3000
y=2000
h=4200

# Esplorazione

## Visualizzazione immagini di esempio

In [ ]:
# Visualizzazione immagini componenti con e senza difetti

counter = 0
for i in CATEGORIES:
    counter = 0
    print(f'CATEGORIA IMMAGINI: {i}')
    path=os.path.join(DIR_ORGINAL_DATA,i)

    for img in os.listdir(path):
        counter += 1
        if counter < 4:
            img_array=imread(os.path.join(path,img))
            print(img_array.shape)
            plt.imshow(img_array)
            plt.show()
        else:
            break

## Analisi caratteristiche immagini

In [ ]:
# Visualizzazione di un immagine difettosa selezionata a campione per l'analisi delle sue caratteristiche

img_path = '../Data/Original/NOK/NOK (2).bmp'
img = cv2.imread(img_path) # Si osservano 3 canali colore. Per default imread legge l'immagine mantenendo la presenza di 3 canali colore (nel caso di immagini in scala di grigi, i tre canali presentano la stessa identica matrice)
print(img.shape)
plt.imshow(img)
plt.show()

In [ ]:
# canale blu
img[:, :, 0]

In [ ]:
# canale verde
img[:, :, 1]

In [ ]:
# canale rosso
img[:, :, 2]

In [ ]:
# Visualizzazione di un immagine difettosa selezionata a campione per l'analisi delle sue caratteristiche, utilizzando il parametro cv2.IMREAD_GRAYSCALE

img_path = '../Data/Original/NOK/NOK (2).bmp'
img = cv2.imread(img_path)

img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # tramite l'utilizzo di cv2.COLOR_BGR2GRAY si ottiene un solo canale colore di output
print(img.shape)
plt.imshow(img, cmap='gray')
plt.show()

## Analisi della distribuzione dei valori di intensità dei pixel

In [ ]:
unique, counts = np.unique(img, return_counts=True)
print(np.asarray((unique, counts)).T)

# I valori dei pixel variano da 0 a 255 dove 0 rappresenta il nero assoluto e 255 il bianco assoluto.

## Verifica class imbalance

In [ ]:
bar_chart_dif_non_dif = {"category": [],
                         "values": []}

# Crezione di una lista category-value
cat_value_list = []

# Definizione della funzione count_images
def count_images(input_folder, filename, filetype):
    count_images = 0
    for i, img in enumerate(glob.glob(input_folder + f"/*.{filetype}"), 1):
        # Conteggio del numero di immagini contenute in ciascuna cartella
        count_images += 1
    cat_value_list.append([filename, count_images])
    print(f"NUMBER OF IMAGES IN FOLDER {filename}:", count_images)
    return cat_value_list

count_images(DIR_ORGINAL_N0K_DATA,
             "NOK",
             "bmp")

count_images(DIR_ORGINAL_0K_DATA,
             "OK",
             "bmp")

for element in cat_value_list:
    bar_chart_dif_non_dif["category"].append(element[0])
    bar_chart_dif_non_dif["values"].append(element[1])

# Conversione del dizionario in dataframe
bar_chart_dif_non_dif = pd.DataFrame(bar_chart_dif_non_dif)

# Riordinamento dei valori
bar_chart_dif_non_dif.sort_values(by=['values'], 
                              ascending=False, 
                              inplace = True)

# Visualizzazione dell'istogramma
fig = px.bar(bar_chart_dif_non_dif, x= 'category', y = 'values')
fig.show()

# Come si può osservare il dataset risulta essere sbilanciato.

# Processing delle immagini

In [ ]:
# Le operazioni eseguite sono quelle di:
#       - conversione in bianco e nero
#       - cropping dell'immagine
#       - ridimensionamento dell'immagine

for i in CATEGORIES:
    print(f'CATEGORY: {i}')
    path = os.path.join(DIR_ORGINAL_DATA,i)

    for img in os.listdir(path):
      print('sto processando una immagine...')
      # Selezione immagine singola
      img_raw = imread(os.path.join(path, img))
      # Conversione in bianco e nero
      img_gray = cv2.cvtColor(img_raw, cv2.COLOR_BGR2GRAY)
      # Cropping
      img_crop = img_gray[x:w, y:h]
      # Ridimensionamento
      width1 = int(img_crop.shape[1] * SCALE_PERCENT / 100)
      height1 = int(img_crop.shape[0] * SCALE_PERCENT / 100)
      dim1 = (width1, height1)
      img_resized = cv2.resize(img_crop, dim1, interpolation = cv2.INTER_AREA)
      # Visualizzazione
      plt.imshow(img_resized)
      # Salvataggio della singola immagine preprocessata nella cartella prepared
      cv2.imwrite(os.path.join(DIR_PREPARED_DATA,i,img), img_resized)
      print('ho salvato la nuova immagine')

# Split train test image form

In [ ]:
splitfolders.ratio(DIR_PREPARED_DATA,
                   output= DIR_TRAIN_TEST_DATA,
                   seed = RANDOM_STATE,
                   ratio = (.65, .35)) # training, testing

# Data Augmentation

In [ ]:
# Creazione di un numero maggiore delle immagini di training tramite l'operazione di Data Augmentation.

train_labels = os.listdir(DIR_TRAIN_DATA)
print(train_labels)

for training_name in train_labels:
    if training_name == 'NOK':  # cartella contenente le immagini difettose
      directory = os.path.join(DIR_TRAIN_DATA, training_name)
      print(directory)
      print(training_name)
      for x in os.listdir(directory):
          file = directory + "/" + str(x)
          image = cv2.imread(file)
          image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
          print(str(x), image.shape)

          # data augmentation
          image_h_flip = utils.horizontal_flip(image, True)
          image_v_flip = utils.vertical_flip(image, True)

          for i in range(30, 38):
              image_rotation_i = utils.rotation(image, i)
              # Salvataggio delle immagini
              cv2.imwrite(f"{str(DIR_TRAIN_DATA)}/{str(training_name)}/{str(x)}_rotation_{i}.bmp", image_rotation_i)

          # Salvataggio delle immagini
          cv2.imwrite(f"{str(DIR_TRAIN_DATA)}/{str(training_name)}/{str(x)}_h_flip.bmp", image_h_flip)
          cv2.imwrite(f"{str(DIR_TRAIN_DATA)}/{str(training_name)}/{str(x)}_v_flip.bmp", image_v_flip)
    else: # cartella contenente le immagini non difettose
        directory = os.path.join(DIR_TRAIN_DATA, training_name)
        for x in os.listdir(directory):
          file = directory + "/" + str(x)
          image = cv2.imread(file)
          image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
          print(str(x), image.shape)

          for i in range(30, 31):
              image_rotation_i = utils.rotation(image, i)
              cv2.imwrite(f"{str(DIR_TRAIN_DATA)}/{str(training_name)}/{str(x)}_rotation_{i}.bmp", image_rotation_i)

In [ ]:
# Controllo del numero di nuovi elementi creati per cartella

for training_name in train_labels:
    count_element = 0
    directory = os.path.join(DIR_TRAIN_DATA, training_name)
    for x in os.listdir(directory):
        count_element += 1
    print(f"Count of element for the directory {training_name}:", count_element)

# Conversione in forma vettoriale

## Train

In [ ]:
target_TRAIN = []
flat_data_TRAIN = []
images_TRAIN = []

# TRAINING

for i in CATEGORIES:
    print("Category is:",i,"\tLabel encoded as:",CATEGORIES.index(i))
    target_class = CATEGORIES.index(i)
    path = os.path.join(DIR_TRAIN_DATA,i)

    for img in os.listdir(path):
        img_array = imread(os.path.join(path,img))
        print(img_array.shape)
        flat_data_TRAIN.append(img_array.flatten())
        target_TRAIN.append(target_class)

# Conversione della lista in un numpy array
flat_data_TRAIN = np.array(flat_data_TRAIN)
target_TRAIN = np.array(target_TRAIN)

# Creazione del dataframe
df_train= pd.DataFrame(flat_data_TRAIN)
df_train = pd.DataFrame(
    scaler.fit_transform(df_train),
    index=df_train.index
)

df_train['Target'] = target_TRAIN
X_train = df_train.iloc[:,:-1] # input data
y_train = df_train.iloc[:,-1] # output data

In [ ]:
X_train

# Ci sono 150 righe che corrispondono alle 150 immagini.
# Le colonne sono invece tutti i pixel flatten.

In [ ]:
y_train  # E' l'etichetta che mostra se l'immagine è difettosa o meno.

## Test

In [ ]:
target_TEST = []
flat_data_TEST = []
images_TEST = []

for i in CATEGORIES:
    print("Category is:",i,"\tLabel encoded as:",CATEGORIES.index(i))
    target_class = CATEGORIES.index(i)
    path = os.path.join(DIR_TEST_DATA,i)

    for img in os.listdir(path):
        img_array = imread(os.path.join(path,img))
        flat_data_TEST.append(img_array.flatten())
        target_TEST.append(target_class)

# Conversione della lista in un numpy array
flat_data_TEST = np.array(flat_data_TEST)
target_TEST = np.array(target_TEST)

# Creazione del dataframe
df_test = pd.DataFrame(flat_data_TEST)
df_test = pd.DataFrame(
    scaler.transform(df_test),
    index=df_test.index
)

df_test['Target'] = target_TEST
X_test = df_test.iloc[:,:-1] # input data
y_test = df_test.iloc[:,-1] # output data

In [ ]:
# Verifica delle dimensioni dei dataframe

print("Dimensions of input training data   (X_train):",  X_train.shape)
print("Dimensions of output training data: (y_train)",   y_train.shape)

print("Dimensions of input testing data:   (X_test)",    X_test.shape)
print("Dimensions of output testing data:  (y_test)",    y_test.shape)

# Data persistence

In [ ]:
# Salvataggio dello scaler

with open('../Models/minMax_scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [ ]:
# Salvataggio di train e test set.

df_train.to_pickle('../Data/Prepared/train_prep.pkl.zip', compression='zip')
df_test.to_pickle('../Data/Prepared/test_prep.pkl.zip', compression='zip')